In [ ]:
import csv,os,glob
output_dir = 'batch20/output_subset/'
aligned_csv = 'pp_aligned.csv'
align_file = os.path.join(output_dir,aligned_csv)

# make a list of the original files -- these should be without any extension
original_files = ['Urine_StrokeDrugs_105_T10_POS','Urine_StrokeDrugs_18_T10_POS','Urine_StrokeDrugs_58_T10_POS']

original_files = glob.glob(os.path.join(output_dir,'*.mgf'))
original_files = [o.split(os.sep)[-1].split('.')[0] for o in original_files]
print(original_files)

original_csvs = [os.path.join(output_dir,original_file + '_quant.csv') for original_file in original_files]
aligned_peaks = []
f_idx_dict =  {}

Load the aligned peaks

In [ ]:
with open(align_file,'r') as f:
    reader = csv.reader(f)
    heads = next(reader)
    file_bits = heads[3:]
    for o in original_files:
        temp = [1 if f.startswith(o) else 0 for f in file_bits]
        f_idx_dict[o] = temp.index(1)
    for line in reader:
        align_id = int(line[0])
        align_mz = float(line[1])
        align_rt = float(line[2])
        intensities = tuple([float(a) for a in line[3:-1]])
        aligned_peaks.append((align_id,align_mz,align_rt,intensities))


In [ ]:
intensities = np.array([list(a[3]) for a in aligned_peaks])
import pylab as plt
%matplotlib inline
plt.figure(figsize=(10,10))
plt.imshow(np.log(intensities+1),aspect='auto')
plt.colorbar()

In [ ]:
trunc_intensities = intensities * (intensities > 5e5)
plt.figure(figsize=(10,10))
plt.imshow(np.log(trunc_intensities+1),aspect='auto')

Match the local peaks (i.e. from the original csvs) to the aligned ones

- Done based upon intensity as there should be exact intensity matches

In [ ]:
matches = {}
for file_pos,o in enumerate(original_files):
    with open(original_csvs[file_pos],'r') as f:
        reader = csv.reader(f)
        heads = next(reader)
        local_peaks = []
        for line in reader:
            id = int(line[0])
            mz = float(line[1])
            rt = float(line[2])
            intensity = float(line[3])
            local_peaks.append((id,mz,rt,intensity))
    local_peaks.sort(key = lambda x: x[3], reverse = True)
    this_idx = f_idx_dict[o]
    aligned_peaks.sort(key = lambda x: x[3][this_idx],reverse = True)
    local_pos = 0
    global_pos = 0
    while local_pos < len(local_peaks) and local_peaks[local_pos][3] > 0:
        # find the match in the global pos
        while abs(aligned_peaks[global_pos][3][this_idx] - local_peaks[local_pos][3]) > 1:
            global_pos += 1
        if not aligned_peaks[global_pos] in matches:
            matches[aligned_peaks[global_pos]] = {}
        matches[aligned_peaks[global_pos]][o] = local_peaks[local_pos]
        local_pos += 1


In [ ]:
# write an output - don't need to run this unless you want the links in a file
output_file = os.path.join(output_dir,'align_links.csv')
with open(output_file,'w') as f:
    heads = ['align ID','row m/z','row retention time'] + original_files
    writer = csv.writer(f)
    writer.writerow(heads)
    for aligned_peak in matches:
        new_row = [aligned_peak[0],aligned_peak[1],aligned_peak[2]]
        for o in original_files:
            val = matches[aligned_peak].get(o,None)
            if val:
                new_row.append(val[0])
            else:
                new_row.append('null')
        writer.writerow(new_row)
        

Load the spectra

In [ ]:
spectra = {}
from MS2 import load_mgf
for o in original_files:
    mgf_file = os.path.join(output_dir,o+'.mgf')
    spectra[o] = load_mgf(mgf_file)
    print("Loaded {} spectra from {}".format(len(spectra[o]),o))


Compute similarity between aligned peaks

In [ ]:
lines = []
from scoring_functions import fast_cosine
for aligned_peak in matches:
    for i in range(len(original_files))[:-1]:
        for j in range(i+1,len(original_files)):
            file_i = original_files[i]
            file_j = original_files[j]
            if file_i in matches[aligned_peak] and file_j in matches[aligned_peak]:
                id_i = matches[aligned_peak][file_i][0]
                id_j = matches[aligned_peak][file_j][0]
                spec_i = spectra[file_i].get(id_i,None)
                spec_j = spectra[file_j].get(id_j,None)
                if spec_i and spec_j:
                    sc,_ = fast_cosine(spec_i,spec_j,0.2,1)
#                     print(file_i,file_j,spec_i.rt,spec_j.rt,sc)
                    lines.append((file_i,file_j,60*matches[aligned_peak][file_i][2],60*matches[aligned_peak][file_j][2],sc))
                    print(file_i,file_j,60*matches[aligned_peak][file_i][2],60*matches[aligned_peak][file_j][2],sc)


In [ ]:
filtered = filter(lambda x: x[4] > 0.9,lines)
a,b,_,_,_ = zip(*filtered)
a = list(a) + list(b)
c = set(a)
counts = []
for d in c:
    counts.append((d,a.count(d)))
counts.sort(key = lambda x: x[1],reverse = True)
print(counts)

Write out a new mzTab file with corrected RTs

In [ ]:
reference_file = 'Pool107_03'

for fi in original_files:        
    print(fi)
    if fi == reference_file:
        continue
    # train the model
    file_1 = reference_file
    file_2 = fi
    subset = []
    min_cosine = 0.9
    for line in lines:
        if line[4] < min_cosine:
            continue
        if line[0] == file_1 and line[1] == file_2:
            subset.append(line)
        elif line[1] == file_1 and line[0] == file_2:
            new_line = (line[1],line[0],line[2],line[3],line[4])
            subset.append(new_line)
    _,_,t1,t2,cos = zip(*subset)
    t1 = np.array(t1)
    t2 = np.array(t2)
    kernel = GPy.kern.RBF(input_dim=1, variance=3., lengthscale=67.)
    X = t2[:,None]
    Y = (t1-t2)[:,None]
    m = GPy.models.GPRegression(X,Y,kernel)
    m.plot()
    with open(os.path.join(output_dir,file_2+'_pp.mzTab'),'r') as f:
        with open(os.path.join('batch20/sub_new_files/'+file_2+'_pp.mzTab'),'w') as g:
            for line in f:
                line = line.rstrip()
                if line.startswith('SMH'):
                    tokens = line.split()
                    rt_pos = [tokens.index('retention_time')]
                    rt_pos.append(tokens.index('opt_assay[1]_peak_rt'))
                if not line.startswith('SML'):
                    g.write(line + '\n')
                else:
                    tokens = line.split()
                    old_rt = 60.0*float(tokens[rt_pos[0]])
                    new_rt,_ = m.predict(np.array([[old_rt]]))
                    new_rt = new_rt[0][0]
                    new_rt += old_rt
                    new_rt/=60.0
                    tokens[rt_pos[0]] = new_rt
                    tokens[rt_pos[1]] = new_rt
                    new_line = '\t'.join([str(t) for t in tokens])
                    g.write(new_line + '\n')



In [ ]:
found = 0
for i in range(len(aligned_peaks)-1):
    p_i = aligned_peaks[i]
    for j in range(i+1,len(aligned_peaks)):
        p_j = aligned_peaks[j]
        if abs(p_i[1] - p_j[1]) < 0.001:
            if abs(p_i[2] - p_j[2]) < 5./60.0:
                for k in range(3):
                    print(p_i[k],p_j[k])
                i_1 = p_i[3]
                i_2 = p_j[3]
                for k in range(len(i_1)):
                    print(i_1[k],i_2[k])
                found +=1
                print()
                print()
    if found > 5:
        break